In [12]:
class PredictiveParser:
    def __init__(self, JackFileLoc):
        self.JackPath = JackFileLoc
        a = open(self.JackPath, "r")
        self.Code = a.readlines()
        self.pureCode = []
        self.kw = ['class','constructor','function','method','field','static','var','int','char','boolean','void','let','do','if','else','while','return','Array','String','Keyboard','screen']
        self.symbols = ['{','}','(',')','[',']','.',',',';','"']
        self.op = ['+','-','*','/','&','|','<','>','=']
        self.unop = ['~','-']
        self.integerconstant = [str(temp) for temp in range(0,3279)]
        self.keywordconstant = ['true','false','null','this']
        className = self.kw[0]
        self.stringconstants = ['Main']
        #make a new file uncommented code.txt

    def rmvecmnts(self, commentedCode): 
        a = open("UnCommentedCode.txt","w")
        p = 0
        for i in commentedCode:
            commentedCode[p] = i.split("//")[0]
            p += 1
        for i in commentedCode:
            commentedCode[commentedCode.index(i)] = i.split("/*")[0]
        for i in commentedCode:
            commentedCode[commentedCode.index(i)] = i.split("*/")[0]
        for i in commentedCode:
            commentedCode[commentedCode.index(i)] = i.split("*")[0]
        #concate the list to a string
        self.concode = "".join(commentedCode)
        a.write(self.concode)
        self.codeString = "\n"
        self.codeString = self.codeString.join(commentedCode)
        self.codeString = self.codeString.replace("{","\n{\n")
        self.codeString = self.codeString.replace("}","\n}\n")
        self.codeString = self.codeString.replace(";","\n;")
        self.codeString = self.codeString.replace(".","\n.\n")
        self.codeString = self.codeString.replace("(","\n(\n")
        self.codeString = self.codeString.replace(")","\n)\n")
        self.codeString = self.codeString.split()
        self.finalcode = "\n"
        self.finalcode = self.finalcode.join(self.codeString)
        
        self.nocomcode = []
        self.nocomcode = self.concode.split("\n")
        #print(self.nocomcode)
    def methods(self):
        kalyan = self.codeString
        self.method = []
        for i in kalyan:
            if i == "method":
                a = kalyan.index(i)
                b = a+2
                self.method.append(kalyan[b])
                kalyan[a] = "method "
        #print(self.method)
    def dotprocessing(self):
        self.tokenizeddots = []
        self.splittingdots =[]

        for i in self.codeString:
            if "." in i:
                self.splittingdots.append(i.split("."))
                self.splittingdots[-1].insert(1, '.')
            else:
                self.splittingdots.append(i)

    def xmltokenizer(self):
        self.tokenedcode = []
        
        temp1 =0
        for words in self.codeString:
                for keywords in self.kw:
                    if words == keywords:
                        self.tokenedcode.append("<keyword>" + words + "</keyword>")
                    else:
                        temp1 +=1
                for keywords in self.symbols:
                    if words == keywords:
                        self.tokenedcode.append("<symbol>" + words + "</symbol>")
                    else:
                        temp1 +=1
                for keywords in self.stringconstants:
                    if words == keywords:
                        self.tokenedcode.append("<stringConstant>" + words + "</stringConstant>")
                    else:
                        temp1 +=1
                for keywords in self.integerconstant:
                    if words == keywords:
                        self.tokenedcode.append("<intConstant>"+ words+"</intConstant>")
                    else:
                        temp1 +=1
                for keywords in self.op:
                    if words == keywords:
                        self.tokenedcode.append("<uniaryoperator>" + words + "</uniaryoperator>")
                    else:
                        temp1 +=1
                for keywords in keywords:
                    if words not in self.kw and words not in self.symbols and words not in self.stringconstants and words not in self.op and words not in self.integerconstant:
                        self.tokenedcode.append("<identifier>" + words + "</identifier>")
                    else:
                        temp1 +=1
        #print(self.tokenedcode)
        #write it to an xml file
        a = open("tokenizedcode.xml","w")
        a.write("<tokens>\n")
        for i in self.tokenedcode:
            a.write(i + "\n")
        a.write("</tokens>\n")

    def semicoloncheck(self):
        #we check if there is a semicolon at the end of the line in self.nocomcode
        for i in self.nocomcode:
            #if there is let or var or do or return it should have semicolon
            if "let" in i or "var" in i or "do" in i or "return" in i:
                if ";" not in i:
                    print("Error: Missing semicolon at the end of line")
                    #print that line
                    print(i)
            if "while" in i:
                if ";" in i:
                    #print it should not have semicolon
                    print("Error: Semicolon should be only present at the end of line and no semicolon at while loop")
                    print(i)

    def docasecheck(self):
        for i in self.nocomcode:
            if "DO" in i:
                print("Error not valid do statement")
                print(i)
                print("Suggestion : use do in small case")
            if "dO" in i:
                print("Error not valid do statement")
                print(i)    
                print("Suggestion : use do in small case")
            if "Do" in i:
                print("Error not valid do statement")
                print(i)  
                print("Suggestion : use do in small case")

    def symbolcheck(self):
        self.expressioncount =0
        for i in self.codeString:
            if i == '{':
                self.expressioncount += 1
            elif i == '}':
                self.expressioncount -= 1
            elif i == '[':
                self.expressioncount += 1
            elif i == ']':
                self.expressioncount -= 1
            elif i == '(':
                self.expressioncount += 1
            elif i == ')':
                self.expressioncount -= 1
        if self.expressioncount == 0:
            print("Symbols OK")
        elif self.expressioncount > 0:
            print("Symbols not closed")
        elif self.expressioncount < 0:
            print("Symbols not opened")

    def ifcheck(self):
        self.ifcount = 0
        self.elsecount = 0
        self.endifcount = 0
        sskif = 0
        sskelse = 0
        self.iflock = 0
        self.elselock = 0
        for i in self.codeString:
            if i == 'if':
                self.ifcount += 1
        for i in self.codeString:
            #the syntax of if is if (expression) statement
            if i == 'if':
                a = self.codeString.index("if")
                sskif = a
                for j in self.codeString[a + 1 : ]:
                    sskif += 1
                    if j == ")":
                        #print(self.codeString[a:sskif])
                        self.iflock +=1
                        self.codeString[a] = "if "
                        break
                #self.elsecount += 1
        for i in self.codeString:
            if i == 'else':
                b = self.codeString.index("else")
                sskelse = b
                for j in self.codeString[b + 1 : ]:
                    sskelse += 1
                    if j == "}":
                        #print(self.codeString[b:sskelse])
                        self.elselock +=1
                        self.codeString[b] = "else "
                        break
                self.elsecount += 1

    def whilecheck(self):
        self.whilecount = 0
        sskwhile = 0
        self.whileloc= 0
        #the syntax of while is while(expression) where expression should not be operators
        for i in self.codeString:
            if i == 'while':
                a = self.codeString.index("while")
                sskwhile = a
                if self.codeString[self.codeString.index(i) + 2] == ")":
                    print("Error: Not a valid while statement")
                    #print that while statement with the error
                    print(self.codeString[self.codeString.index(i) : self.codeString.index(i) + 3])
                    print("Suggestion : give some while condition")
                    self.whilecount -= 1
                    self.codeString[self.codeString.index(i)] = "while "
                    break
                for j in self.codeString[a + 1 :]:
                    sskwhile += 1
                    if j == "{":
                        #print(self.codeString[a:sskwhile - 1])
                        self.whilecount += 1
                        self.codeString[a] = "while "
                        break
                

    def docheck(self):
        self.do = 0
        sskdowhile = 0
        self.doloc = 0
        do = self.codeString
        a = self.codeString.index("do")
        sskdowhile = a
        #the syntax of do is do statement while(expression)
        for i in self.codeString:
            if i == 'do':
                if do[do.index(i)+1] not in self.method and do[do.index(i)+2] != ".":
                    print("Error at ")
                    self.do += 1
                    #print that line
                    for j in do[a + 1 : ]:
                        sskdowhile += 1
                        if j == ";":
                            print(do[a:sskdowhile])
                            self.doloc +=1
                            do[a] = "do "
                            break
    def returncheck(self):
        self.returncount = 0
        sskreturn = 0
        self.returnloc = 0
        #the syntax of return is return expression
        for i in self.codeString:
            if i == 'return':
                a = self.codeString.index("return")
                sskreturn = a
                for j in self.codeString[a + 1 :]:
                    sskreturn += 1
                    if j == ";":
                        #print(self.codeString[a:sskreturn - 1])
                        self.returncount += 1
                        self.codeString[a] = "return "
                        break
    def var(self):
        self.vardecloc = 0
        ssk = 0
        var = self.codeString
        self.varlist = []
        for i in var:
            b = var[var.index(i)+2]
            if (i == "var" ) :
                #index of var
                varin = var.index(i)
                #add varname to varlist
                self.varlist.append(var[varin+2])
                
                if b in self.kw  :
                    print("Error: Variable name cannot be a keyword")
                    a = var.index("var")
                    ssk = a
                    for j in var[a + 1 : ]:
                        ssk += 1
                        if j == ";":
                            print(var[a:ssk])
                            self.vardecloc +=1
                            var[a] = "var "
                            break
                        self.vardecloc +=1
                else:
                    a = var.index("var")
                    ssk = a
                    for j in var[a + 1 : ]:
                        ssk += 1
                        if j == ";":
                            #print(let[a:ssk])
                            self.vardecloc +=1
                            var[a] = "var "
                            break                    
    def let(self):
        self.jlock = 0
        ssk = 0
        let = self.codeString
        for i in let:
            b = let[let.index(i)+1]
            if (i == "let"):
                # if self.symbols is present in b throw error and in self.varlist
                if b in self.op :
                    print("Let statement cannot have symbols after it")
                    a = let.index("let")
                    ssk = a
                    for j in let[a + 1 : ]:
                        ssk += 1
                        if j == ";":
                            print(let[a:ssk])
                            self.jlock +=1
                            let[a] = "let "
                            break                   
                        self.jlock +=-1
                if b not in self.varlist:
                    print("Error: Variable name not declared")
                    a = let.index("let")
                    ssk = a
                    for j in let[a + 1 : ]:
                        ssk += 1
                        if j == ";":
                            print(let[a:ssk])
                            self.jlock +=1
                            let[a] = "let "
                            break                    
                        self.jlock +=-1
                else:
                    a = let.index("let")
                    ssk = a
                    for j in let[a + 1 : ]:
                        ssk += 1
                        if j == ";":
                            self.jlock +=1
                            let[a] = "let "
                            break               
        print(self.codeString)
    def main(self):
        self.rmvecmnts(self.Code)
        self.dotprocessing()
        self.semicoloncheck()
        self.xmltokenizer()
        self.symbolcheck()
        self.var()
        self.let()
        self.ifcheck()
        self.whilecheck()
        self.methods()
        self.docheck()
        self.docasecheck()
        self.returncheck()
        print("XML code generated")

In [13]:
a = PredictiveParser("Main.jack")

a.main()

Symbols OK
Error: Variable name not declared
['let', 'sum', '=', '0']
Error: Variable name not declared
['let', 'sum', '=', 'sum', '+', 'a[i]']
['class', 'Main', '{', 'function', 'void', 'main', '(', ')', '{', 'var ', 'Array', 'a', ';', 'var ', 'int', 'length', ';', 'var ', 'int', 'i', ';', 'let ', 'length', '=', 'Keyboard', '.', 'readInt', '(', '"HOW', 'MANY', 'NUMBERS?', '"', ')', ';', 'let ', 'a', '=', 'Array', '.', 'new', '(', 'length', ')', ';', 'let ', 'i', '=', '0', ';', 'while', '(', ')', '{', 'let ', 'a', '[i]', '=', 'Keyboard', '.', 'readInt', '(', '"ENTER', 'THE', 'NEXT', 'NUMBER:', '"', ')', ';', 'let ', 'i', '=', 'i', '+', '1', ';', '}', 'let ', 'i', '=', '0', ';', 'let ', 'sum', '=', '0', ';', 'while', '(', 'i', '<', 'length', ')', '{', 'let ', 'sum', '=', 'sum', '+', 'a[i]', ';', 'let ', 'i', '=', 'i', '+', '1', ';', '}', 'do', 'Output', '.', 'printString', '(', '"THE', 'AVERAGE', 'IS:', '"', ')', ';', 'do', 'Output', '.', 'printInt', '(', 'sum', '/', 'length', ')', ';',